Install necessary packages

In [ ]:
#!pip install transformers spacy scikit-learn nltk pandas
#!pip install torch
#!python3 -m spacy download en_core_web_md
#nltk.download('punkt')
#!pip install xformers
#!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f

#!pip install accelerate
#!pip install bitsandbytes
#!pip install transformers
#!pip install ipywidgets

# local LLM - Falcon 7b

In [ ]:
# notebook was run on Ubuntu in WSL
# ensure WSL is version 2 to access CUDA hardware

#notebook ran with these libraries and versions
#accelerate                0.23.0
#bitsandbytes              0.41.1
#bitsandbytes-cuda116      0.26.0.post2
#ipywidgets                8.1.1
#nvidia-cublas-cu11        11.10.3.66
#nvidia-cuda-cupti-cu11    11.7.101
#nvidia-cuda-nvrtc-cu11    11.7.99
#nvidia-cuda-runtime-cu11  11.7.99
#nvidia-cudnn-cu11         8.5.0.96
#nvidia-cufft-cu11         10.9.0.58
#nvidia-curand-cu11        10.2.10.91
#nvidia-cusolver-cu11      11.4.0.1
#nvidia-cusparse-cu11      11.7.4.91
#nvidia-nccl-cu11          2.14.3
#nvidia-nvtx-cu11          11.7.91
#tokenizers                0.13.3
#torch                     2.0.1
#transformers              4.33.3

# to get nvidia libraries, follow install guide at:
# https://developer.nvidia.com/cuda-downloads

In [1]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
    #, load_in_8bit_fp32_cpu_offload=True,
    #llm_int8_enable_fp32_cpu_offload=True
)

In [3]:
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
model_id = "vilsonrodrigues/falcon-7b-sharded"

In [4]:
device_map={
    "transformer.word_embeddings": "cpu",
    "transformer.word_embeddings_layernorm": 0,
    "lm_head": "cpu",
    "transformer.h": 0,
    "transformer.ln_f": 0,
}
device_map = "auto"

In [5]:
model_4bit = AutoModelForCausalLM.from_pretrained(
    model_id,
    #device_map=device_map,
    quantization_config=quantization_config
    , trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
#print(model_4bit)

FalconForCausalLM(
  (transformer): FalconModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x FalconDecoderLayer(
        (self_attention): FalconAttention(
          (maybe_rotary): FalconRotaryEmbedding()
          (query_key_value): Linear4bit(in_features=4544, out_features=4672, bias=False)
          (dense): Linear4bit(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): FalconMLP(
          (dense_h_to_4h): Linear4bit(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear4bit(in_features=18176, out_features=4544, bias=False)
        )
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)


In [8]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=296,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [9]:
sequences = pipeline("Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other \
animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:")

In [11]:
print(sequences)

[{'generated_text': "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron: <giggle>\nDaniel:...\nGirafatron: Hi, Daniel!\nDaniel:...\nDaniel (thinking): Oh god!\nGirafatron: Daniel, I just want you know that I love you, and you're the bestest best friend in the whole wide world.\nDaniel: Girafatron, why are you wearing my head?\nGirafatron: <giggle> Daniel you are the mostest bestestest friend I've ever had!\nDaniel: Oh, thanks.\nGirafatron: <giggle>\nDaniel: Girafatron?\nGirafatron: What's it, Daniel?\nDaniel: What are you doing with your hand?\nGirafatron: <giggle>\nDaniel: Girafatron?\nGirafatron: <giggle>\nDaniel: Girafatron!\nGirafatron: Daniel what? <giggle>\nDaniel: Girafatron what are you"}]


# Other

In [ ]:
import nltk
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import string
import json
import string
import random
import transformers
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch import nn
from transformers import BitsAndBytesConfig
from transformers import pipeline
import main

In [ ]:
save_pretrained

In [ ]:
temp=main.calculate_prompts("input.json")

In [ ]:
main.save_json(temp)

In [ ]:
main.split_data()

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    #load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Train

In [ ]:
!pip3 install -q -U trl

In [ ]:
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
data = load_dataset("timdettmers/openassistant-guanaco")

In [ ]:
model_name = "tiiuae/falcon-7b"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    dataset_text_field="text",
    max_seq_length=512,
)

In [ ]:
trainer.train()

In [ ]:
trainer.train()

In [ ]:
import torch, einops
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments
)
from peft.tuners.lora import LoraLayer

from trl import SFTTrainer


def create_and_prepare_model():
    compute_dtype = getattr(torch, "float16")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        "tiiuae/falcon-7b", quantization_config=bnb_config, device_map={"": 0}, trust_remote_code=True
    )

    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ],
    )

    tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    return model, peft_config, tokenizer


training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=1000,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)

model, peft_config, tokenizer = create_and_prepare_model()
model.config.use_cache = False
dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)

trainer.train()

In [ ]:
with open("train_data.json", "r") as json_file:
    training_data = json.load(json_file)

input_texts = main.generate_input_texts(training_data)

# Initialize the falcon-7b-sharded model and tokenizer
model_name = "vilsonrodrigues/falcon-7b-sharded"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the input texts
encoded_input = tokenizer(
    input_texts,
    return_tensors="pt",
    padding="max_length",
    max_length=512  # Adjust max_length as needed
)
print(len(encoded_input))

In [ ]:
# Load the training data from the JSON file
with open("train_data.json", "r") as json_file:
    training_data = json.load(json_file)

input_texts = main.generate_input_texts(training_data)

In [ ]:
# Initialize the falcon-7b-sharded model and tokenizer
model_name = "vilsonrodrigues/falcon-7b-sharded"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
# Tokenize the input texts
encoded_input = tokenizer(
    input_texts,
    return_tensors="pt",
    padding="max_length",
    max_length=512  # Adjust max_length as needed
)

In [ ]:
#input_ids = main.truncate(encoded_input)
input_ids = encoded_input

# Create a DataLoader for training
dataset = main.TextDataset(input_ids)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [ ]:
# Training configuration
num_epochs = 5
learning_rate = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
for epoch in range(num_epochs):
    total_loss = 0
    model.train()

    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
        for key in batch:
            batch[key] = batch[key].to(device)

        optimizer.zero_grad()
        outputs = model(**batch, labels=batch["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(dataloader)}")

# Save the trained model
model.save_pretrained("trained_falcon_model")
tokenizer.save_pretrained("trained_falcon_model")

print("Training completed. Model saved as 'trained_falcon_model'.")


In [ ]:
# Load the trained falcon-7b-sharded model and tokenizer
model_path = "trained_falcon_model"  # Replace with the path to your trained model
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set the device for inference (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a user prompt for text generation
user_prompt = "Please provide recommendations for places to visit in"

# Generate text based on the user prompt
generated_text = model.generate(
    input_ids=tokenizer.encode(user_prompt, return_tensors="pt").to(device),
    max_length=100,  # Adjust as needed for desired text length
    num_return_sequences=1,  # Number of sequences to generate
    temperature=0.7,  # Adjust for randomness (higher values make text more random)
    top_k=50,  # Adjust to control the diversity of generated text
    top_p=0.95,  # Adjust to control the diversity of generated text
    pad_token_id=50256,  # GPT-2 token for padding
    eos_token_id=50256,  # GPT-2 token for end of sequence
    bos_token_id=50256,  # GPT-2 token for beginning of sequence
)

# Decode and print the generated text
generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)


In [ ]:
input_dim = 10
embedding_dim = 2
embedding = nn.Embedding(input_dim, embedding_dim)
err = True
if err:
    #Any input more than input_dim - 1, here input_dim = 10
    #Any input less than zero
    input_to_embed = torch.tensor([10])
else:
    input_to_embed = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
embed = embedding(input_to_embed)
print(embed)

In [ ]:
!pip3 install bitsandbytes-cuda110 bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
import transformers
import torch

#model_name = "ybelkada/falcon-7b-sharded-bf16"
#model_name = "tiiuae/falcon-7b"
#model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"
model_name = "vilsonrodrigues/falcon-7b-sharded"

bnb_config = BitsAndBytesConfig(
    #load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

In [ ]:
sequences = pipeline(
   "Write a poem about Valencia.",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


In [ ]:
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM

dataset = load_dataset("imdb", split="train")

model_id = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
)
trainer.train()
